# In diesem Notebook wird unser Datawarehouse erstellt. Wir lesen die Daten aus den CSV-Dateien ein und wandeln die so erstellten Tabellen in ein Sternschema um.

#### Importieren der von uns benötigten Funktionen

In [130]:
import sqlite3
import pandas as pd

#### Erstellen einer Verbindung zur Datenbank

In [131]:
#connect to DB
conn = sqlite3.connect('datawarehouse.db')
cursor = conn.cursor()
print("Opened database successfully");

Opened database successfully


#### Mithilfe von Pandas CSV-Dateien einlesen

In [132]:
#load CSVs
careplans = pd.read_csv('Daten/careplans.csv', sep=",")
conditions = pd.read_csv('Daten/conditions.csv', sep=",")
disease = pd.read_csv('Daten/disease.csv', sep=",")
immunizations = pd.read_csv('Daten/immunizations.csv', sep=",")
medications = pd.read_csv('Daten/medications.csv', sep=",")
observations = pd.read_csv('Daten/observations.csv', sep=",")
patients = pd.read_csv('Daten/patients.csv', sep=",")

#### Vorhandene Tabellen löschen, um Konflikte zu vermeiden

In [133]:
#Delete Tables in case they exist
cursor.execute("DROP TABLE IF EXISTS careplans")
cursor.execute("DROP TABLE IF EXISTS conditions")
cursor.execute("DROP TABLE IF EXISTS disease")
cursor.execute("DROP TABLE IF EXISTS immunizations")
cursor.execute("DROP TABLE IF EXISTS medications")
cursor.execute("DROP TABLE IF EXISTS observations")
cursor.execute("DROP TABLE IF EXISTS patients")
cursor.execute("DROP TABLE IF EXISTS facts_table")


#### Erstellen der Tabellen

In [134]:
#Create Tables
cursor.execute('''CREATE TABLE IF NOT EXISTS careplans (
                           Id STRING PRIMARY KEY,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           REASONCODE STRING,
                           REASONDESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 
                       )
                       ''')


cursor.execute('''CREATE TABLE IF NOT EXISTS conditions (
                           condition_code INTEGER PRIMARY KEY AUTOINCREMENT,
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 
                        )
                       ''')



cursor.execute('''CREATE TABLE IF NOT EXISTS observations (
                           observation_code INTEGER PRIMARY KEY AUTOINCREMENT,
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           VALUE STRING,
                           UNITS STRING,
                           TYPE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       )
                       ''')



cursor.execute('''CREATE TABLE IF NOT EXISTS patients (
                           Id STRING PRIMARY KEY,
                           BIRTHDATE DATE,
                           DEATHDATE DATE,
                           SSN STRING,
                           DRIVERS STRING,
                           PASSPORT STRING,
                           PREFIX STRING,
                           FIRST STRING,
                           LAST STRING,
                           SUFFIX STRING,
                           MAIDEN STRING,
                           MARITAL STRING,
                           RACE STRING,
                           ETHNICITY STRING,
                           GENDER STRING,
                           BIRTHPLACE STRING,
                           ADDRESS STRING,
                           CITY STRING,
                           STATE STRING,
                           COUNTY STRING,
                           ZIP STRING,
                           LAT INTEGER,
                           LON INTEGER,
                           HEALTHCARE_EXPENSES INTEGER,
                           HEALTHCARE_COVERAGE INTEGER
                       )
                       ''')

cursor.execute('''CREATE TABLE IF NOT EXISTS immunizations(
                           immunization_code INTEGER PRIMARY KEY AUTOINCREMENT,
                           DATE DATE,
                           PATIENT STRING,
                           ENCOUNTER STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           BASE_COST INTEGER,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients (Id) 
                           FOREIGN KEY (Encounter)
                              REFERENCES encounters (Id) 

                       )
                       ''')






#### Die Daten aus den Dataframes in die erstellten Tabellen schreiben

In [135]:
#Insert into table
careplans.to_sql('careplans', conn, if_exists='append', index=False)
conditions.to_sql('conditions', conn, if_exists='append', index=False)
immunizations.to_sql('immunizations', conn, if_exists='append', index=False)
medications.to_sql('medications', conn, if_exists='append', index=False)
observations.to_sql('observations', conn, if_exists='append', index=False)
patients.to_sql('patients', conn, if_exists='append', index=False)


1050

# Sternschema

Der Grund für die Verwendung eines Sternschemas liegt darin, die Anzahl der Join-Bedingungnen für die Auswertung der einzelnen Tabellen zu reduzieren. bei dem Sternschema wird eine Zentrale Datenbank erzeugt, welche die relevanten Daten enthält. Um den Kern des Sternschemas werden weitere Dimensions Tabellen angeordnet. Wichtig dabei kann es zu duplizierungen der Daten in kommen.  Das Sternschema erlaubt uns, uns die Tabellen passende für unsere Forschungsfrage zusammenzujoinen. 

### Faktentabelle erstellen

Die Faktentabelle enthält die Primary Keys aus allen anderen Tabellen. Zusätzlich werden die Attribute VALUE, START_DATE und STOP_DATE hinzugefügt. VALUE enthält hierbei entweder den CODE oder den VALUE der Tabelle aus der die Daten stammen. Das Selbe gilt für START_DATE und STOP_DATE. Gibt es in der ursprünglichen Tabelle nur ein DATE und kein START_DATE und STOP_DATE wird das Datum als START_DATE übernommen und STOP_DATE bleibt dann leer.

In [136]:
cursor.execute('''CREATE TABLE IF NOT EXISTS facts_table (
                          patient_ID STRING,
                          careplan_ID INT,
                          condition_id INT,
                          immunization_code INT,
                          VALUE STRING,
                          START_DATE DATE,
                          STOP_DATE DATE,
                          observation_code INT, 
                          FOREIGN KEY (patient_ID)
                            REFERENCES patients (Id) 
                          FOREIGN KEY (careplan_ID)
                            REFERENCES careplans (Id) 
                          FOREIGN KEY (condition_id)
                            REFERENCES conditions (condition_code)
                          FOREIGN KEY (immunization_code)
                            REFERENCES immunizations (immunization_code) 
                          FOREIGN KEY (observation_code)
                            REFERENCES observations (observation_code)                             

                       )
                       ''')

In [137]:
print(pd.read_sql_query("PRAGMA table_info('facts_table')", conn))

   cid               name    type  notnull dflt_value  pk
0    0         patient_ID  STRING        0       None   0
1    1        careplan_ID     INT        0       None   0
2    2       condition_id     INT        0       None   0
3    3  immunization_code     INT        0       None   0
4    4              VALUE  STRING        0       None   0
5    5         START_DATE    DATE        0       None   0
6    6          STOP_DATE    DATE        0       None   0
7    7   observation_code     INT        0       None   0


### Daten in Faktentabelle übertragen

In [138]:
cursor.execute('''INSERT INTO facts_table        
                    (patient_ID, careplan_ID, VALUE, START_DATE, STOP_DATE) 
                    SELECT PATIENT, Id, CODE, START, STOP
                    FROM careplans
                    ;''')



cursor.execute('''INSERT INTO facts_table        
                    (patient_ID, observation_code, VALUE, START_DATE) 
                    SELECT PATIENT, CODE, VALUE, DATE 
                    FROM OBSERVATIONS
                    ;''')


cursor.execute('''INSERT INTO facts_table        
                    (patient_ID, condition_Id, VALUE, START_DATE, STOP_DATE) 
                    SELECT PATIENT, condition_code, CODE, START, STOP
                    FROM conditions
                    ;''')

cursor.execute('''INSERT INTO facts_table        
                    (patient_ID, immunization_code, VALUE, START_DATE) 
                    SELECT PATIENT, immunization_code, CODE, DATE
                    FROM immunizations
                    ;''')



#### Nach dem Bearbeiten müssen wir unsere Anpassungen commiten und die Verbindung zur Datenbank beenden

In [139]:
conn.commit()
conn.close()

In [140]:
print("Datawarehouse created successfully")

Datawarehouse created successfully
